# Assignment 2: Building a Simple Index

In this assignment, we will build a simple search index, which we will use later for Boolean retrieval. The assignment tasks are again at the bottom of this document.

## Loading the Data

In [1]:
Summaries_file = 'data/cognition_Summaries.pkl.bz2'
Abstracts_file = 'data/cognition_Abstracts.pkl.bz2'

In [2]:
import pickle, bz2
from collections import namedtuple

Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )

paper = namedtuple( 'paper', ['title', 'authors', 'year', 'doi'] )

for (id, paper_info) in Summaries.items():
    
    Summaries[id] = paper( *paper_info )
    
Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )

Let's have a look at what the data looks like for an example of a paper:

In [3]:
Summaries[28102890]

paper(title='Handgrip Strength in Old and Very Old Adults: Mood, Cognition, Function, and Mortality.', authors=['Stessman J', 'Rottenberg Y', 'Fischer M', 'Hammerman-Rozenberg A', 'Jacobs JM'], year=2017, doi='10.1111/jgs.14509')

In [4]:
Abstracts[28102890]

'To determine the trajectory of handgrip strength (HGS) from age 70 to 90 and its association with mood, cognition, functional status, and mortality.'

## Some Utility Functions

We'll define some utility functions that allow us to tokenize a string into terms, perform linguistic preprocessing on a list of terms, as well as a function to display information about a paper in a nice way. Note that these tokenization and preprocessing functions are rather naive. We will improve them in a later assignment.

In [5]:
def tokenize(text):
    """
    Function that tokenizes a string in a rather naive way. Can be extended later.
    """
    return text.split(' ')

def preprocess(tokens):
    """
    Perform linguistic preprocessing on a list of tokens. Can be extended later.
    """
    result = []
    for token in tokens:
        result.append(token.lower())
    return result

print(preprocess(tokenize("Lorem ipsum dolor sit AMET")))

['lorem', 'ipsum', 'dolor', 'sit', 'amet']


In [6]:
from IPython.display import display, HTML
import re

def display_summary( id, show_abstract=False, show_id=True, extra_text='' ):
    """
    Function for printing a paper's summary through IPython's Rich Display System.
    Trims long author lists, and adds a link to the paper's DOI (when available).
    """
    s = Summaries[id]
    lines = []
    title = s.title
    if s.doi != '':
        title = '<a href=http://dx.doi.org/{:s}>{:s}</a>'.format(s.doi, title)
    title = '<strong>' + title + '</strong>'
    lines.append(title)
    authors = ', '.join( s.authors[:20] ) + ('' if len(s.authors) <= 20 else ', ...')
    lines.append(str(s.year) + '. ' + authors)
    if (show_abstract):
        lines.append('<small><strong>Abstract:</strong> <em>{:s}</em></small>'.format(Abstracts[id]))
    if (show_id):
        lines.append('[ID: {:d}]'.format(id))
    if (extra_text != ''):
         lines.append(extra_text)
    display( HTML('<br>'.join(lines)) )

display_summary(28102890)
display_summary(28102890, show_abstract=True)

## Creating our first index

We will now create an _inverted index_ based on the words in the titles and abstracts of the papers in our dataset. We will implement our inverted index as a Python dictionary with term strings as keys and posting lists (implemented as Python lists) as values. We include all the tokens we can find in the title and (if available) in the abstract:

In [7]:
from collections import defaultdict

inverted_index = defaultdict(list)

# This can take a few seconds:
for id in sorted(Summaries.keys()):
    term_set = set(preprocess(tokenize(Summaries[id].title)))
    if id in Abstracts:
        term_set.update(preprocess(tokenize(Abstracts[id])))
    for term in term_set:
        inverted_index[term].append(id)

Let's see what's in the index for the example term 'amsterdam':

In [8]:
print(inverted_index['amsterdam'])

[10547144, 17888546, 18289505, 19004529, 20594617, 20969529, 21482441, 22049191, 23232034, 23802509, 23964225, 25114679, 25598195, 25631781, 25684214, 25877267, 26976043, 27148107, 28462387, 28671504, 29029762, 29209183, 29406610, 29843832, 29848469, 31026198, 31082289, 31547145, 31730165, 33008394, 34203041, 34408904, 34541288, 34925184, 35149261]


We can now use this inverted index to answer simple one-word queries, for example to show all papers that contain the word 'utrecht':

In [9]:
query_word = 'utrecht'
for i in inverted_index[query_word]:
    display_summary(i, show_abstract=True)

----------

# Tasks

**Your name:** Ryan Ott

### Task 1

Implement the function `and_merge` outlined below. This function takes two posting lists from the index that can be assumed to be sorted already, and it should return the result of the merging of the two lists with AND. The resulting list should therefore include all the elements that appear in both lists. As explained on the slides, this operation should take advantage of the input lists being sorted already, should not perform any additional sorting operation, and should go through each of the input lists just once. Then, test your function with an example.

In [10]:
def and_merge(pos1, pos2):
    result = []
    i = j = 0
    while i < len(pos1) and j < len(pos2):
        if pos1[i] == pos2[j]:
            result.append(pos1[i])
            i += 1
            j += 1
        elif pos1[i] < pos2[j]:
            i += 1
        else:
            j += 1
    return result

# Papers that include the terms "automatic" AND "water"
for paper in and_merge(inverted_index["automatic"], inverted_index["water"]):
    display_summary(paper, show_abstract=True)

### Task 2

Similarly as above, implement the function `or_merge` outlined below that executes an OR merging of the lists. The resulting list should therefore include all the elements that appear in at least one of the lists (note this is an 'inclusive OR' and not a 'XOR'). Again, this operation should take advantage of the input lists being sorted already, should not perform any additional sorting operation, and should go through each of the input lists just once. Elements that appear in both input list should only appear once in the output list. Test your function again with an example.

In [11]:
def or_merge(pos1, pos2):
    result = []
    i = j = 0
    while i < len(pos1) and j < len(pos2):
        if pos1[i] == pos2[j]:
            result.append(pos1[i])
            i += 1
            j += 1
        elif pos1[i] < pos2[j]:
            result.append(pos1[i])
            i += 1
        else:
            result.append(pos2[j])
            j += 1
    result.extend(pos1[i:])
    result.extend(pos2[j:])
    return result

# First 5 papers that include the terms "automatic" OR "water"
for paper in or_merge(inverted_index["automatic"], inverted_index["water"])[:5]:
    display_summary(paper, show_abstract=True)

### Task 3

Construct a function called `and_query` that takes as input a single string, consisting of one or more words, and returns as function value a list of matching documents. `and_query`, as its name suggests, should require that all query terms are present in the documents of the result list.

For that, access the variable `inverted_index` from above and use the method `and_merge` that you defined. Also use the `tokenize` and `preprocess` functions we defined above to tokenize and preprocess your query.

Again demonstrate the working of your function with an example (choose one that leads to fewer than 100 hits to not overblow this notebook file).

In [12]:
def and_query(query):
    result = []
    for term in preprocess(tokenize(query)):
        if term not in inverted_index: return []
        if result == []:
            result = inverted_index[term]
        else:
            result = and_merge(result, inverted_index[term])
    return result

test_queries = ["automATic WATer", "automATic WATer cowabunga"]  # First should show same papers as in task 1, second none
for query in test_queries:
    print("Query: {:s}".format(query))
    print("-----------------------------------------------------------------")
    for paper in and_query(query):
        display_summary(paper)

Query: automATic WATer
-----------------------------------------------------------------


Query: automATic WATer cowabunga
-----------------------------------------------------------------


### Task 4

Construct another function called `or_query` that works in the same way as `and_query` you just implemented, but returns as function value the documents that contain _at least one_ of the words in the query, using the `or_merge` function you defined.

Demonstrate the working of this function also with an example (again, choose one that leads to fewer than 100 hits).

In [13]:
def or_query(query):
    result = []
    for term in preprocess(tokenize(query)):
        if term in inverted_index:
            result = or_merge(result, inverted_index[term])
    return result

test_queries = ["metereological", "metereological Zagreb", "metereological Zagreb watER"]  # First should no results, second only 2 including "zagreb", third should show first 10 results of ("water" + query 2 results)
for query in test_queries:
    print("Query: {:s}".format(query))
    print("-----------------------------------------------------------------")
    for paper in or_query(query)[:10]:
        display_summary(paper)

Query: metereological
-----------------------------------------------------------------
Query: metereological Zagreb
-----------------------------------------------------------------


Query: metereological Zagreb watER
-----------------------------------------------------------------


### Task 5

Why does `and_query('adult handgrip strength')` not return our example paper 28102890, even though it mentions handgrip strength and adults in the abstract? (You do not have to implement anything to fix this yet!)

Block 1 below shows the results of and_query("adult handgrip strength"): it returns two papers, but not the one from the example.

However, when we adapt the query to and_query("adults: handgrip strength") we get (more) different results, including our example paper.

This is because the preprocessing and tokenisation functions used are very primitive and don't yet handle punctuation or stemming (see the term list in block 3 of the adapted query).

In [14]:
print("Block 1")
for paper in and_query("adult handgrip strength"):
    display_summary(paper)
print("---------------------------------------------------------------------------")
print("Block 2")
for paper in and_query("adults: handgrip strength"):
    display_summary(paper)
print("---------------------------------------------------------------------------")
print("Block 3")
print(preprocess(tokenize("adults: handgrip strength")))

Block 1


---------------------------------------------------------------------------
Block 2


---------------------------------------------------------------------------
Block 3
['adults:', 'handgrip', 'strength']


# Submission

Submit the answers to the assignment via Canvas as a modified version of this Notebook file (file with `.ipynb` extension) that includes your code and your answers.

Before submitting, restart the kernel and re-run the complete code (**Kernel > Restart & Run All**), and then check whether your assignment code still works as expected.

Don't forget to add your name, and remember that the assignments have to be done **individually**, and that code sharing or copying are **strictly forbidden** and will be punished.